<a href="https://colab.research.google.com/github/Soedirman-Machine-Learning/experiment-repo/blob/master/final_implementation_of_SLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf

model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', input_shape=(224, 224, 3), padding='same'))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
#model.add(tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))


#model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
#model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
#model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
#model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))


#model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
#model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
#model.add(tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3), strides=(1,1), activation='relu', padding='same'))
#model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(tf.keras.layers.Flatten())
#model.add(tf.keras.layers.Dense(50, activation="relu"))
#model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(32, activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(8, activation="softmax"))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)    

In [0]:
from keras.utils import to_categorical
import numpy as np
from PIL import Image
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from numpy import argmax
from keras.callbacks import EarlyStopping, ModelCheckpoint
from skimage import img_as_ubyte
import skimage.transform
import time
from keras import backend as K


from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.15, zoom_range=0.2, brightness_range=[0.8, 1.3], horizontal_flip=False)
 #Labels go here

path1 = '/content/drive/My Drive/SLR/Images/' 
listing = os.listdir(path1)    
count = 0


for epoc in range(10):
  S=[0,0,0,0,0,0,0,0]
  X=[]  #Images goes here
  Y=[] 
  print('ginga', epoc)
  for file in listing:
    #print(ord(file[3])-48,file[3])
    count+=1
    if ord(file[3])-48 == epoc:
      #print(file)
      S[ord(file[0])-65] += 1
      image = cv2.imread(path1+file)

      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      image = img_as_ubyte(skimage.transform.resize(image, (224, 224)))    
      image=np.array(image)
      image.astype(np.float32)
      image = image/255
      X.append(image)
      Y.append(ord(file[0])-65)
      start = time.time()

      img = np.expand_dims(image, 0)
      aug_itr = gen.flow(img)
      aug_images = [next(aug_itr)[0].astype(np.float32) for i in range(35)]
      for i in aug_images:
        i = i/255
        X.append(i)
        Y.append(ord(file[0])-65)
      end = time.time()
      #print(len(Y)," ",(end-start),Y[len(Y)-1])
  print(S)
  
  X=np.array(X)
  Y=np.array(Y)

  Y = Y.reshape(-1,1)
  Y = to_categorical(Y)

  print(X.shape,Y.shape)

  x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

  model.compile(optimizer ='adam', loss = "categorical_crossentropy", metrics = ["accuracy"])

  model.fit(x_train,y_train,validation_data=(x_test,y_test), verbose=1,epochs=10,batch_size=50,shuffle=True)
  np.delete(X,X)
  np.delete(X,X)
#,callbacks=[monitor,checkpointer]
tf.keras.models.save_model(model, '/content/my_model.h5')




ginga 0


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


[5, 6, 5, 6, 8, 5, 8, 7]
(1800, 224, 224, 3) (1800, 8)
Train on 1620 samples, validate on 180 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1620/1620 [==============================] - 48s 30ms/sample - loss: 2.3792 - acc: 0.1506 - val_loss: 2.0363 - val_acc: 0.2833
Epoch 2/10
1620/1620 [==============================] - 33s 21ms/sample - loss: 1.7604 - acc: 0.3179 - val_loss: 1.3394 - val_acc: 0.5944
Epoch 3/10
1620/1620 [==============================] - 33s 21ms/sample - loss: 1.4281 - acc: 0.4000 - val_loss: 1.1578 - val_acc: 0.5611
Epoch 4/10
1620/1620 [==============================] - 33s 20ms/sample - loss: 1.3105 - acc: 0.4506 - val_loss: 0.9790 - val_acc: 0.7000
Epoch 5/10
1620/1620 [==============================] - 33s 20ms/sample - loss: 1.2366 - acc: 0.4691 - val_loss: 0.8187 - val_acc: 0.6889
Epoch 6/10
1620/1620 [==============================] - 33s 21ms/sample - loss: 1.2558 - acc: 0.4512 - val_loss: 0.7051 - val_acc: 0.8056
Epoch 7/10
1620/1620 [

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


ginga 1
[5, 5, 5, 6, 7, 5, 8, 7]
(1728, 224, 224, 3) (1728, 8)
Train on 1555 samples, validate on 173 samples
Epoch 1/10
1555/1555 [==============================] - 35s 23ms/sample - loss: 1.2408 - acc: 0.4714 - val_loss: 0.7188 - val_acc: 0.8497
Epoch 2/10
1555/1555 [==============================] - 32s 20ms/sample - loss: 1.0968 - acc: 0.5190 - val_loss: 0.6595 - val_acc: 0.9249
Epoch 3/10
1555/1555 [==============================] - 32s 21ms/sample - loss: 1.0722 - acc: 0.5113 - val_loss: 0.4964 - val_acc: 0.9133
Epoch 4/10
1555/1555 [==============================] - 32s 20ms/sample - loss: 1.0801 - acc: 0.5151 - val_loss: 0.5462 - val_acc: 0.9191
Epoch 5/10
1555/1555 [==============================] - 32s 20ms/sample - loss: 0.9822 - acc: 0.5479 - val_loss: 0.4828 - val_acc: 0.9191
Epoch 6/10
1555/1555 [==============================] - 32s 20ms/sample - loss: 1.0149 - acc: 0.5498 - val_loss: 0.5196 - val_acc: 0.9191
Epoch 7/10
1555/1555 [==============================] - 32s 21

In [0]:
from keras.models import Sequential
from sklearn.preprocessing import OneHotEncoder
from numpy import argmax
from keras.utils import to_categorical
from keras.layers import Flatten,Dense,Input,AveragePooling2D
import json
import numpy as np
from PIL import Image
import os
import cv2
import numpy
from keras.applications import mobilenet 
from skimage import img_as_ubyte
import skimage.transform
import tensorflow as tf

model2 = tf.keras.models.load_model("/content/my_model.h5")

pred = model2.predict(x_test)
su=0 
for i in range (len(x_test)):
  if np.argmax(pred[i]) == np.argmax(y_test[i]):
    su+=1
print(su/len(y_test))
    
  
  #print(chr(pred[i]+65),chr(np.argmax(y_test[i])+65))

0.996742671009772


In [0]:
from tensorflow.contrib import lite
converter = lite.TFLiteConverter.from_keras_model_file( '/content/my_model.h5' ) # Your model's name
model = converter.convert()
file = open( '/content/model10.tflite' , 'wb' ) 
file.write( model )

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 16 variables.
INFO:tensorflow:Converted 16 variables to const ops.


30276464

In [0]:
#below codes are obsolete

In [0]:
from keras.utils import to_categorical
import numpy as np
from PIL import Image
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from numpy import argmax
from keras.callbacks import EarlyStopping, ModelCheckpoint
from skimage import img_as_ubyte
import skimage.transform
import time
from keras import backend as K


from keras.preprocessing.image import ImageDataGenerator

gen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.15, zoom_range=0.2, brightness_range=[0.8, 1.3], horizontal_flip=False)
 #Labels go here
S=[]
X=[]  #Images goes here
Y=[] 
path1 = '/content/drive/My Drive/SLR/Images/' 
listing = os.listdir(path1)    
count = 0
for file in listing:
 
  count+=1
  if file.endswith('.jpg'):
    image = cv2.imread(path1+file)
    #print(file)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #image.resize(224,224,3)
    #image.reshape(224,224,3)
    image = img_as_ubyte(skimage.transform.resize(image, (224, 224)))
    #cv2.resize(image,(224,224))
    #plot = plt.imshow(image)
    
    

    #image = image.astype(np.uint8)
    #S.append(image)
    
    image=np.array(image)
    image.astype(np.float32)
    image = image/255
    X.append(image)
    Y.append(ord(file[0])-65)
    start = time.time()

    img = np.expand_dims(image, 0)
    aug_itr = gen.flow(img)
    aug_images = [next(aug_itr)[0].astype(np.float32) for i in range(5)]
    for i in aug_images:
      #i = i.astype(np.uint8)
      #i = img_as_ubyte(skimage.transform.resize(i, (224, 224)))
      #i = i.astype(np.float32)
      i = i/255
      X.append(i)
      Y.append(ord(file[0])-65)
    end = time.time()
    print(len(Y)," ",(end-start),Y[len(Y)-1])
X=np.array(X)
Y=np.array(Y)

#print(model.layers[len(mobile_model.layers)])

Y = Y.reshape(-1,1)
Y = to_categorical(Y)

print(X.shape,Y.shape)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



model.compile(optimizer ='adam', loss = "categorical_crossentropy", metrics = ["accuracy"])

model.fit(x_train,y_train,validation_data=(x_test,y_test), verbose=1,epochs=10,batch_size=50,shuffle=True) 
#,callbacks=[monitor,checkpointer]
tf.keras.models.save_model(model, '/content/my_model.h5')




Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


9   0.13520288467407227 7
18   0.1335277557373047 7
27   0.12703371047973633 4
36   0.135528564453125 5
45   0.14671802520751953 5
54   0.13726401329040527 4
63   0.14076995849609375 0
72   0.15035676956176758 6
81   0.12377429008483887 2
90   0.14816617965698242 2
99   0.15196990966796875 6
108   0.14446401596069336 0
117   0.14791226387023926 6
126   0.15519928932189941 4
135   0.1421213150024414 5
144   0.14244318008422852 5
153   0.15189862251281738 7
162   0.14565181732177734 4
171   0.15041041374206543 7
180   0.1423354148864746 4
189   0.14599251747131348 7
198   0.1377730369567871 7
207   0.15798044204711914 0
216   0.1567249298095703 5
225   0.1614058017730713 7
234   0.1525719165802002 5
243   0.15891575813293457 4
252   0.153458833694458 6
261   0.1518247127532959 4
270   0.15167999267578125 6
279   0.1534438133239746 0
288   0.1549227237701416 6
297   0.15840411186218262 2
306   0.15149831771850586 2
315   0.15197372436523438 6
324   0.15957117080688477 0
333   0.1492395401

In [0]:
from keras.models import Sequential
from sklearn.preprocessing import OneHotEncoder
from numpy import argmax
from keras.utils import to_categorical
from keras.layers import Flatten,Dense,Input,AveragePooling2D
import json
import numpy as np
from PIL import Image
import os
import cv2
import numpy
from keras.applications import mobilenet 
from skimage import img_as_ubyte
import skimage.transform
import tensorflow as tf


image = cv2.imread('/content/drive/My Drive/SLR/Images/H_21.jpg')
#image=np.array(image)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

image = img_as_ubyte(skimage.transform.resize(image, (224, 224)))

#image.resize(1,224,224,3)
image = image.astype(np.float32)
image=image/255
#print(len(model2.layers))

model2 = tf.keras.models.load_model("/content/my_model.h5")

image = image.reshape(1,224,224,3)

pred = model2.predict(image)
print(pred,np.argmax(pred))
    
  
  #print(chr(pred[i]+65),chr(np.argmax(y_test[i])+65))

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


[[3.3415600e-24 1.9920531e-22 2.1985350e-14 3.6958926e-24 6.9170402e-32
  2.1366334e-20 5.2140038e-03 9.9478602e-01]] 7
